## How to create parallelized collections

In [1]:
#Create a list of 10000 integers
data = range(1, 10001)

#It will just print out the function but not data itself
print(data)

#Let's use SparkContext and parallelize() function to create a RDD Dataset and spread it across the cluster partitions
ds_rdd = sc.parallelize(data, 5)
ds_rdd_1 = sc.parallelize([('a',1),('a',2),('b',4),('b',1),('c',3),('c',3)])
ds_rdd_2 = sc.parallelize([('a',[1,2,3,4]),('b',[5,6,7])])
ds_rdd_3 = sc.parallelize(range(1, 11))
ds_rdd_4 = sc.parallelize([('b',1),('c',3),('c',3)])

#Collect() action will trigger execution and then we can print 10 first element of our list
print(ds_rdd.collect()[:10])
print(ds_rdd_1.collect())
print(ds_rdd_2.collect())
print(ds_rdd_3.collect())

range(1, 10001)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[('a', 1), ('a', 2), ('b', 4), ('b', 1), ('c', 3), ('c', 3)]
[('a', [1, 2, 3, 4]), ('b', [5, 6, 7])]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## Using Help command...

In [2]:
help(sc.parallelize)

Help on method parallelize in module pyspark.context:

parallelize(c, numSlices=None) method of __main__.RemoteContext instance
 Distribute a local Python collection to form an RDD. Using xrange
 is recommended if the input represents a range for performance.
 
 >>> sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()
 [[0], [2], [3], [4], [6]]
 >>> sc.parallelize(xrange(0, 6, 2), 5).glom().collect()
 [[], [0], [], [2], [4]]

## Basic informations

In [3]:
#Returns number of partitions
print("Number of RDD partitions: ",ds_rdd.getNumPartitions())
#Returns number of instances
print("Number of instances: ",ds_rdd.count())
#Returns sum of instances
print("Sum of Rdd instances: ",ds_rdd.sum())
#Returns number of instances by key
print("Number of instances by key: ",ds_rdd_1.countByKey())
#Returns number of instances by value
print("Number of instances by value: ",ds_rdd_1.countByValue())
#Returns dictionary with key,value pairs. Some instances are removed if keys are duplicated
print("Dictionary with key,value pairs: ",ds_rdd_1.collectAsMap())

Number of RDD partitions: 5
Number of instances: 10000
Sum of Rdd instances: 50005000
Number of instances by key: defaultdict(<class 'int'>, {'c': 2, 'b': 2, 'a': 2})
Number of instances by value: defaultdict(<class 'int'>, {('a', 1): 1, ('b', 1): 1, ('a', 2): 1, ('c', 3): 2, ('b', 4): 1})
Dictionary with key,value pairs: {'c': 3, 'b': 1, 'a': 2}

## Getting summary stats

In [4]:
#Returns min. value of RDD elements
print("Min. value: ",ds_rdd.min())
#Returns max. value of RDD elements
print("Max. value: ",ds_rdd.max())
#Returns mean. value of RDD elements
print("Mean value: ",ds_rdd.mean())
#Returns standard deviation value of RDD elements
print("Stdev. value: ",ds_rdd.stdev())
#Returns varaiance value of RDD elements
print("Variance value: ",ds_rdd.variance())
#Returns histogram for specified number of bins of RDD elements
print("Histogram: ",ds_rdd.histogram(10))
#Returns overall stats of RDD elements
print("Summary stats.: ",ds_rdd.stats())

Min. value: 1
Max. value: 10000
Mean value: 5000.5
Stdev. value: 2886.75133151
Variance value: 8333333.25
Histogram: ([1.0, 1000.9, 2000.8, 3000.7, 4000.6, 5000.5, 6000.4, 7000.3, 8000.2, 9000.1, 10000], [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000])
Summary stats.: (count: 10000, mean: 5000.5, stdev: 2886.75133151, max: 10000.0, min: 1.0)

# Applying lambda functions/Iterating

In [5]:
#Apply lambda function on each rdd element
print(ds_rdd.map(lambda x: x*2).collect()[:10])
#Apply lambda function on each rdd element and flatten result
print(ds_rdd_1.flatMap(lambda x:x).collect())
#Apply flatMap function on key,value pair without changing key value
print(ds_rdd_2.flatMapValues(lambda x:x).collect())
#create a function and apply it to all rdd elements
def print_el(x): print(x)
ds_rdd.foreach(print_el)


[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
['a', 1, 'a', 2, 'b', 4, 'b', 1, 'c', 3, 'c', 3]
[('a', 1), ('a', 2), ('a', 3), ('a', 4), ('b', 5), ('b', 6), ('b', 7)]

## Selecting/Filtering data

In [6]:
#return a list of all elements of rdd
print(ds_rdd.collect()[:10])
#return a given number of rdd elements
print(ds_rdd.take(10))
#return first element of rdd
print(ds_rdd.first())
#return a given number of top (last) elements
print(ds_rdd.top(5))
#return sampled subset of rdd
print(ds_rdd.sample(withReplacement=False,fraction=0.001, seed=1).collect())
#filter rdd basing on some condition
print(ds_rdd_1.filter(lambda x: 'b' in x).collect())
#return distinct rdd values
print(ds_rdd_1.distinct().collect())
#return list of rdd keys
print(ds_rdd_1.keys().collect())
#return list of rdd values
print(ds_rdd_1.values().collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
1
[10000, 9999, 9998, 9997, 9996]
[2392, 5703, 5790, 7549, 8556]
[('b', 4), ('b', 1)]
[('a', 2), ('b', 4), ('a', 1), ('b', 1), ('c', 3)]
['a', 'a', 'b', 'b', 'c', 'c']
[1, 2, 4, 1, 3, 3]

## Reducing/Grouping

In [7]:
#Merge/reduce rdd values by key
print(ds_rdd_1.reduceByKey(lambda x,y:x+y).collect())
#Merge/reduce rdd values
print(ds_rdd_1.reduce(lambda x,y:x+y))
#return rdd of grouped values
print(ds_rdd_3.groupBy(lambda x: x%2).mapValues(list).collect())
#group rdd by key
print(ds_rdd_1.groupByKey().mapValues(list).collect())

[('a', 3), ('c', 6), ('b', 5)]
('a', 1, 'a', 2, 'b', 4, 'b', 1, 'c', 3, 'c', 3)
[(0, [2, 4, 6, 8, 10]), (1, [1, 3, 5, 7, 9])]
[('a', [1, 2]), ('c', [3, 3]), ('b', [4, 1])]

## Mathematical operations/Sorting

In [8]:
#returns all rdd values avaliable in one rdd and not available in the other one
print(ds_rdd_1.subtract(ds_rdd_4).collect())
#returns all rdd values for which key is avaliable in one rdd and is not available in the other one
print(ds_rdd_1.subtractByKey(ds_rdd_4).collect())
#returm a cartesian product of two rdds
print(ds_rdd_1.cartesian(ds_rdd_4).collect())
#sort rdd by given function
print(ds_rdd_1.sortBy(lambda x: x[1]).collect())
#sort rdd by Key
print(ds_rdd_1.sortByKey().collect())

[('a', 2), ('b', 4), ('a', 1)]
[('a', 1), ('a', 2)]
[(('a', 1), ('b', 1)), (('a', 1), ('c', 3)), (('a', 1), ('c', 3)), (('a', 2), ('b', 1)), (('a', 2), ('c', 3)), (('a', 2), ('c', 3)), (('b', 4), ('b', 1)), (('b', 4), ('c', 3)), (('b', 4), ('c', 3)), (('b', 1), ('b', 1)), (('b', 1), ('c', 3)), (('b', 1), ('c', 3)), (('c', 3), ('b', 1)), (('c', 3), ('c', 3)), (('c', 3), ('c', 3)), (('c', 3), ('b', 1)), (('c', 3), ('c', 3)), (('c', 3), ('c', 3))]
[('a', 1), ('b', 1), ('a', 2), ('c', 3), ('c', 3), ('b', 4)]
[('a', 1), ('a', 2), ('b', 4), ('b', 1), ('c', 3), ('c', 3)]

## Loading file to dataframe/performing queries

In [9]:
#Load csv file to dataframe
df = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load("/FileStore/tables/worldcities.csv")

display(df)

city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
Malishevë,Malisheve,42.4822,20.7458,Kosovo,XK,XKS,Malishevë,admin,null,1901597212
Prizren,Prizren,42.2139,20.7397,Kosovo,XK,XKS,Prizren,admin,null,1901360309
Zubin Potok,Zubin Potok,42.9144,20.6897,Kosovo,XK,XKS,Zubin Potok,admin,null,1901608808
Kamenicë,Kamenice,42.5781,21.5803,Kosovo,XK,XKS,Kamenicë,admin,null,1901851592
Viti,Viti,42.3214,21.3583,Kosovo,XK,XKS,Viti,admin,null,1901328795
Shtërpcë,Shterpce,42.2394,21.0272,Kosovo,XK,XKS,Shtërpcë,admin,null,1901828239
Shtime,Shtime,42.4331,21.0397,Kosovo,XK,XKS,Shtime,admin,null,1901598505
Vushtrri,Vushtrri,42.8231,20.9675,Kosovo,XK,XKS,Vushtrri,admin,null,1901107642
Dragash,Dragash,42.0265,20.6533,Kosovo,XK,XKS,Dragash,admin,null,1901112530
Podujevë,Podujeve,42.9111,21.1899,Kosovo,XK,XKS,Podujevë,admin,null,1901550082


In [10]:
#pring dataframe schema
df.printSchema()

root
-- city: string (nullable = true)
-- city_ascii: string (nullable = true)
-- lat: double (nullable = true)
-- lng: double (nullable = true)
-- country: string (nullable = true)
-- iso2: string (nullable = true)
-- iso3: string (nullable = true)
-- admin_name: string (nullable = true)
-- capital: string (nullable = true)
-- population: double (nullable = true)
-- id: integer (nullable = true)

In [11]:
# Create a temporary view
df.createOrReplaceTempView("worldcities_csv")

In [12]:
%sql
show tables

database,tableName,isTemporary
default,cleaned_taxes,false
,worldcities_csv,true


In [13]:
%sql
/* Query created view*/
select * from `worldcities_csv` where population>10000000
sort by population desc

city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
Tokyo,Tokyo,35.685,139.7514,Japan,JP,JPN,Tōkyō,primary,3.5676E7,1392685764
New York,New York,40.6943,-73.9249,United States,US,USA,New York,null,1.9164071E7,1840034016
Mexico City,Mexico City,19.4424,-99.131,Mexico,MX,MEX,Ciudad de México,primary,1.9028E7,1484247881
Mumbai,Mumbai,19.017,72.857,India,IN,IND,Mahārāshtra,admin,1.8978E7,1356226629
São Paulo,Sao Paulo,-23.5587,-46.625,Brazil,BR,BRA,São Paulo,admin,1.8845E7,1076532519
Delhi,Delhi,28.67,77.23,India,IN,IND,Delhi,admin,1.5926E7,1356872604
Shanghai,Shanghai,31.2165,121.4365,China,CN,CHN,Shanghai,admin,1.4987E7,1156073548
Kolkata,Kolkata,22.495,88.3247,India,IN,IND,West Bengal,admin,1.4787E7,1356060520
Dhaka,Dhaka,23.7231,90.4086,Bangladesh,BD,BGD,Dhaka,primary,1.2797394E7,1050529279
Buenos Aires,Buenos Aires,-34.6025,-58.3975,Argentina,AR,ARG,"Buenos Aires, Ciudad Autónoma de",primary,1.2795E7,1032717330


## Spark in use - Check the most common 10 words in "Pan Tadeusz" poem  (without words shorter than 4 letters)

In [14]:
import re
lines = sc.textFile("/FileStore/tables/pan_tadeusz_ansi.txt")
words = lines.flatMap(lambda x: re.sub(r'[^a-z ]+', '', x.lower().replace('\n',' ')).split(' '))
pairs = words.map(lambda x: (x,1))
count = pairs.reduceByKey(lambda x,y: x+y)
print(count.filter(lambda x: len(x[0])>3).sortBy(lambda x :x[1],False).collect()[:10])


[('lecz', 206), ('jest', 176), ('rzekl', 155), ('tylko', 149), ('jako', 145), ('mnie', 144), ('hrabia', 136), ('sedzia', 133), ('wiec', 112), ('tadeusz', 107)]